# PyTorch Train Process
## 1. 框架使用思路
使用 PyTorch 框架进行训练以及测试的流程：
- 判断设备（GPU or CUP）
- 初始化可视化工具（SummaryWriter）
- 初始化数据集（datasets）
- 加载数据集为可迭代格式（DataLoader）
- 搭建网络（nn.Module）并实例化网络模型或者直接加载模型（load_state_dict）
- 构建损失函数（nn.Loss）
- 初始化优化器（torch.optim）
- 开始训练和测试
- 保存模型（state_dict()）

## 2. 示例代码

In [ ]:
# 网络模型，数据（输入和标注），损失函数可以 .to(device)


import torch
import torchvision
from torch.utils.data import DataLoader
from torch import nn
from collections import OrderedDict
from torch.utils.tensorboard import SummaryWriter


# 判断 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tensorboard
writer = SummaryWriter("./logs_train")

# 数据集
train_data = torchvision.datasets.CIFAR10(root='../moduleLearning/CIFAR10', train=True, transform=torchvision.transforms
                                          .ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root='../moduleLearning/CIFAR10', train=False, transform=torchvision.transforms
                                         .ToTensor(), download=True)
# 数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

# dataloader
train_dataloader = DataLoader(dataset=train_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)


# 网络搭建
class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 32, 5, padding=2)),
            ('maxpool1', nn.MaxPool2d(2)),
            ('conv2', nn.Conv2d(32, 32, 5, padding=2)),
            ('maxpool2', nn.MaxPool2d(2)),
            ('conv3', nn.Conv2d(32, 64, 5, padding=2)),
            ('maxpool3', nn.MaxPool2d(2)),
            ('flatten', nn.Flatten()),
            ('linear1', nn.Linear(1024, 64)),
            ('linear2', nn.Linear(64, 10))
        ]))

    def forward(self, input_data):
        return self.model(input_data)


# 实例化网络模型
model1 = TestModel()
model1_dict = torch.load("./model/model1_199.pth")
model1.load_state_dict(model1_dict)
model1 = model1.to(device)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

# 优化器
learning_rate = 0.0001
optimizer = torch.optim.SGD(model1.parameters(), lr=learning_rate)

# 设置训练参数
# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 100

for i in range(epoch):
    print("------------ 第 {} 轮训练开始 ------------".format(i+1))
    model1.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model1(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数：{}，损失函数：{}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 训练完一轮，开始测试
    model1.eval()
    # 整体测试损失值（直接相加）
    total_test_loss = 0
    # 整体正确率
    total_test_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model1(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            total_test_accuracy += (outputs.argmax(1) == targets).sum()
    print("测试集 Loss：{}".format(total_test_loss))
    print("测试集 Accu：{}".format(total_test_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_test_accuracy/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(model1.state_dict(), "./model/model1_{}.pth".format(i+200))
    print("模型{}已保存".format(i+200))

writer.close()
